In [40]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np 
import time 
import random
import IPython.display as ipd
import pyaudio
import sys
import librosa
import librosa.display
from tqdm import tqdm
from matplotlib import pyplot as plt
import soundfile as sf

sys.path.append('../../')

import pdb
#import utils.audio as a
#import utils.plots as plots
import utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# load (fe)male audio sequences without alpha, beta 
PATH = '../../datasets/freesound.org/wav/'
SR = 22050 


100%|██████████| 383/383 [00:00<00:00, 704.92it/s]


((383,), (51237,), (383,), (46, 2205))

In [14]:
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle

train = False

if train:
    #X_scaled = MinMaxScaler(feature_range=(0, 1)).fit_transform(X)
    X_scaled = chunk_feat_vec.reshape(chunk_feat_vec.shape[0], -1)
    #x_pca = PCA(n_components=2).fit_transform(X_scaled)
    np.random.seed(42)
    #model = hmm.GaussianHMM(n_components=3, covariance_type="diagonal", n_iter=100)
    model = hmm.GaussianHMM(n_components=3, verbose=True, covariance_type="full", n_iter=100)
    #model.fit(np.array(x_pca), len_vec)
    model.fit(np.array(X_scaled), len_vec)

    with open("hmm_feat_20_100ms.pkl", "wb") as file: pickle.dump(model, file)
else:
    with open("hmm_feat_20_100ms.pkl", "rb") as file: model = pickle.load(file)


In [42]:
# load (fe)male audio sequences without alpha, beta 
PATH = '../../datasets/freesound.org/'
SR = 22050 

dataset = []
chunk_sz = int(SR * .1) # 100ms
num_feats = 6
files = os.listdir(PATH + "wav/")
X = []
for f in tqdm(files):
    if f.split('.')[-1] != 'wav':
        continue
    #pdb.set_trace()
    audio, sr = utils.audio.loadAudio(PATH + "wav/" + f, sr=SR)
    chunks = utils.misc.slidingWindow(audio, chunk_sz, chunk_sz)
    X = []
    
    for i, (chunk) in enumerate(chunks):
         # mfcc
        feat = librosa.feature.mfcc(y=chunks[i, :], sr=SR, n_mfcc=num_feats).flatten() # 13x44

        # RMS mag spec
        rms = librosa.feature.rms(y=chunks[i, :]).flatten() #1x44

        # Spectral centroid
        cent = librosa.feature.spectral_centroid(y=chunks[i, :], sr=SR).flatten()#1x44

        # Spectral bandwidth
        band = librosa.feature.spectral_bandwidth(y=chunks[i, :], sr=SR).flatten()#1x44

        # Spectral flatness
        flat = librosa.feature.spectral_flatness(y=chunks[i, :]).flatten()#1x44

        # spectral rolloff
        rolloff = librosa.feature.spectral_rolloff(y=chunks[i, :], sr=SR).flatten()#1x44

        # zero crossing rate 
        zcross = librosa.feature.zero_crossing_rate(chunks[i, :]).flatten()#1x44

        # Spectral contrast
        S = np.abs(librosa.stft(chunks[i, :]))
        spec_contrast = librosa.feature.spectral_contrast(S=S, sr=SR).flatten() #7x44
        
        # concetante to feature vector
        X.append(np.concatenate((feat, rms, cent, band, flat, rolloff, zcross, spec_contrast)))
        
    audio_denoised = utils.audio.removeNoise(model, 0, chunk_sz, np.asarray(X), audio)    
        #print(np.asarray(X).reshape(-1).shape)
        #pdb.set_trace()
    #np.save(PATH + "wav_denoised/" + f, audio_denoised)
    sf.write(PATH + "wav_denoised/" + f, audio_denoised, SR)

100%|██████████| 383/383 [03:02<00:00,  2.10it/s]
